# Outline
1. Import continental and assumed debt for each state
2. Map towns to county names using mapping file + fuzzy string matching
3. Use county shape files to create maps

## Documentation Notes
1. All non-exact matches will be printed out when doing merges (exact matches are not printed)

## Cleaning Questions
1. What do we do about an entry that has entries for some debt types, but not for others? Do we want to drop those entries, impute the values, or set NA values to 0?
2. What do we do about entries that have a state, but no town (ie: State of Connecticut)? What about no state?

In [1]:
import numpy as np
import pandas as pd
from rapidfuzz import process

In [2]:
def combineCols(df, num = 3):
    change_df_agg = pd.DataFrame(columns = ['old', 'new', 'type'])
    for col in ['town', 'state', 'occupation']:
        # " ".join(x.split()) removes all excess whitespace
        # creates set with all unique instances of column
        if num == 3:
            df[col] = [set([" ".join(x.split()) for x in [t1, t2, t3] if not pd.isnull(x)]) for t1, t2, t3 in zip(df[col+'1'],
                                                                                                                  df[col+'2'],
                                                                                                                  df[col+'3'])]
        else:
            df[col] = [set([" ".join(x.split()) for x in [t1, t2] if not pd.isnull(x)]) for t1, t2 in zip(df[col+'1'],
                                                                                                          df[col+'2'])]
        if not any(df[col].apply(lambda x: len(x) > 1).tolist()):
            # change set to string
            print("reformatting {}".format(col))
            df[col] = df[col].apply(lambda x: x.pop() if x != set() else np.nan)
        else:
            print("{} column has multiple unique entries".format(col))
            print("see table at end for new entries")
            # keep the value that has the most characters, otherwise change set to string
            old = df[df[col].apply(lambda x: len(x) > 1)][col]
            df[col] = df[col].apply(lambda x: x.pop() if len(x) == 1 else np.nan if x == set() else max(list(x), key=len))
            # new dataframe to keep track of changes
            # create copy of change_df that removes all duplicates by turning old column, which is
            # of type set to type string
            change_df = pd.DataFrame([old, df.loc[old.index][col]]).T
            change_df.columns = ['old', 'new']
            change_df['type'] = col
            change_df_str = change_df.copy()
            change_df_str['old'] = change_df_str['old'].astype(str)
            change_df_str = change_df_str.drop_duplicates()
            # add filtered database of changes to aggregate dataset
            change_df_agg = pd.concat([change_df_agg, change_df.loc[change_df_str.index]])

    # add functions to combine asset totals, handle missing debt values

    return df[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents','3p_Dollar', '3p_Cents',
               'town', 'state', 'occupation']], change_df_agg

In [3]:
"""ind1, ind2, ind3 = CT_CD[['6p_Dollar', '6p_Cents']].dropna(thresh = 1).index,
                   CT_CD[['6p_def_Dollar', '6p_def_Cents']].dropna(thresh = 1).index,
                   CT_CD[['3p_Dollar', '3p_Cents']].dropna(thresh = 1).index
ind = set(ind1).intersection(ind2).intersection(ind3)"""

"ind1, ind2, ind3 = CT_CD[['6p_Dollar', '6p_Cents']].dropna(thresh = 1).index,\n                   CT_CD[['6p_def_Dollar', '6p_def_Cents']].dropna(thresh = 1).index,\n                   CT_CD[['3p_Dollar', '3p_Cents']].dropna(thresh = 1).index\nind = set(ind1).intersection(ind2).intersection(ind3)"

## Import Data
1. Import CD and ASD for each state, combine the multiple town/state/occupation columns (if they exist) into one
2. Concatenate all the separate datasets into two (one CD and one ASD)

### Connecticut

In [4]:
CD_all = pd.DataFrame(columns = ['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents','3p_Dollar', '3p_Cents',
                                  'town', 'state', 'occupation'])
ASD_all = pd.DataFrame(columns = ['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents','3p_Dollar', '3p_Cents',
                                  'town', 'state', 'occupation'])

In [5]:
# importing desired columns and rename
CT_CD_raw = pd.read_excel("../../Data/Post1790/CT/CT_post1790_CD_ledger.xlsx",
                      header = 13, usecols = 'H, I, J, K, L, N, O, X, Y, Z, AA, AB, AD, AE, AN, AO, AP, AQ, AR, AT, AU')
CT_CD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                     'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                     'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
# unify occupation, town and state columns
CT_CD, change_df = combineCols(CT_CD_raw)
CD_all = pd.concat([CD_all, CT_CD])
change_df

town column has multiple unique entries
see table at end for new entries
reformatting state
reformatting occupation


,old,new,type
526,"{Miford, Milford}",Milford,town
799,"{Nowich, Norwich}",Norwich,town


In [6]:
# importing desired columns and rename
CT_ASD_raw = pd.read_excel("../../Data/Post1790/CT/CT_post1790_ASD_ledger.xlsx",
                       header = 13, usecols = 'H, I, J, K, L, N, O, X, Y, Z, AA, AB, AD, AE, AN, AO, AP, AQ, AR, AT, AU')
CT_ASD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                     'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                     'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
# unify occupation, town and state columns
CT_ASD, change_df = combineCols(CT_ASD_raw)
ASD_all = pd.concat([ASD_all, CT_ASD])
change_df

town column has multiple unique entries
see table at end for new entries
reformatting state
occupation column has multiple unique entries
see table at end for new entries


,old,new,type
125,"{Norwalk, Norwall}",Norwalk,town
811,"{Treasuer 1st Society in Lyme, 1st Society in ...",Treasuer 1st Society in Lyme,occupation


### Georgia
No need to do additional cleaning because there's only one state/city/occupation column

In [7]:
# importing desired columns and rename
# prepare loan dataset
GA_CD_raw = pd.read_excel("../../Data/Post1790/GA/T694_GA_Loan_Office_CD.xlsx",
                      header = 10, usecols = 'Q, R, S, T, U, Z, AA, AB, AC, AD, AE')
GA_CD_raw.columns = ['First Name', 'Last Name', 'town', 'state', 'occupation', '6p_Dollar', '6p_Cents',
                 '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']
GA_CD = GA_CD_raw[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents','3p_Dollar', '3p_Cents',
                   'town', 'state', 'occupation']]
CD_all = pd.concat([CD_all, GA_CD])

### Maryland

In [8]:
#prepare loan dataset
MD_CD_raw = pd.read_excel("../../Data/Post1790/MD/MD_post1790_CD.xlsx",
                      header = 11, usecols = 'G, H, I, J, K, L, M, U, V, W, X, Y, Z, AA, AI, AJ, AK, AL, AM, AN, AO')
MD_CD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                 'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                 'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
# unify occupation, town and state columns
MD_CD , change_df = combineCols(MD_CD_raw)
CD_all = pd.concat([CD_all, MD_CD])
change_df

reformatting town
reformatting state
reformatting occupation


,old,new,type


In [9]:
#prepare loan dataset
MD_ASD_raw = pd.read_excel("../../Data/Post1790/MD/MD_post1790_ASD.xlsx",
                      header = 11, usecols = 'G, H, I, J, K, L, M, U, V, W, X, Y, Z, AA, AI, AJ, AK, AL, AM, AN, AO')
MD_ASD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                 'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                 'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
# unify occupation, town and state columns
MD_ASD , change_df = combineCols(MD_ASD_raw)
ASD_all = pd.concat([ASD_all, MD_ASD])
change_df

reformatting town
reformatting state
reformatting occupation


,old,new,type


### North Carolina

In [10]:
NC_CD_raw = pd.read_excel("../../Data/Post1790/NC/T695_R4_NC_CD.xlsx",
                      header = 11, usecols = 'J, K, L, M, N, W, X, Z, AA, AC, AD ')
NC_CD_raw.columns = ['First Name', 'Last Name', 'town', 'state', 'occupation', '6p_Dollar', '6p_Cents',
                 '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']
NC_CD = NC_CD_raw[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents','3p_Dollar', '3p_Cents',
                   'town', 'state', 'occupation']]
CD_all = pd.concat([CD_all, NC_CD])

In [11]:
NC_ASD_raw = pd.read_excel("../../Data/Post1790/NC/T695_R3_NC_ASD.xlsx", header = 10, usecols = 'H, I, J, K, L, P, Q, R, S, T, U')
NC_ASD_raw.columns = ['First Name', 'Last Name', 'town', 'state', 'occupation',  '6p_Dollar', '6p_Cents',
                      '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']
NC_ASD = NC_ASD_raw[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents','3p_Dollar', '3p_Cents',
                         'town', 'state', 'occupation']]
ASD_all = pd.concat([ASD_all, NC_ASD])

### New Hampshire

In [12]:
#prepare loan dataset
NH_CD_raw = pd.read_excel("../../Data/Post1790/NH/T652_R6_New_Hampshire_CD.xlsx",
                      header = 10, usecols = 'I, J, K, L, M, N, O, P, Q, R, S')
NH_CD_raw.columns = ['First Name', 'Last Name', 'town', 'state', 'occupation', '6p_Dollar', '6p_Cents',
                     '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents']
NH_CD = NH_CD_raw[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents','3p_Dollar', '3p_Cents',
                   'town', 'state', 'occupation']]
CD_all = pd.concat([CD_all, NH_CD])

In [13]:
NH_ASD_raw = pd.read_excel("../../Data/Post1790/NH/T652_New_Hampshire_ASD.xlsx", header = 12,
                       usecols = 'G, H, I, J, K, M, N, V, W, X, Y, Z, AA, AB, AK, AL, AM, AN')
NH_ASD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                      'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                      'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
# unify occupation, town and state columns
NH_ASD, change_df = combineCols(NH_ASD_raw, 2)
ASD_all = pd.concat([ASD_all, NH_ASD])
change_df

reformatting town
reformatting state
reformatting occupation


,old,new,type


### New York
Doesn't have town/occupation/state

In [14]:
"""#prepare loan dataset
NY_CD_raw = pd.read_excel("../../Data/Post1790/NY/NY_1790_CD.xlsx",
                      header = 11, usecols = 'H, I, M, N, X, Y, AC, AD, AM, AN, AR, AS')
NY_CD_raw.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents',
                 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                 'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
NY_CD_raw['state'] = np.nan
NY_CD_raw['town'] = np.nan
NY_CD_raw['occupation'] = np.nan
NY_CD = NY_CD_raw[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents','3p_Dollar', '3p_Cents',
                   'town', 'state', 'occupation']]
CD_all = pd.concat([CD_all, NY_CD])
"""

'#prepare loan dataset\nNY_CD_raw = pd.read_excel("../../Data/Post1790/NY/NY_1790_CD.xlsx",\n                      header = 11, usecols = \'H, I, M, N, X, Y, AC, AD, AM, AN, AR, AS\')\nNY_CD_raw.columns = [\'First Name\', \'Last Name\', \'6p_Dollar\', \'6p_Cents\',\n                 \'First Name.1\', \'Last Name.1\', \'6p_def_Dollar\', \'6p_def_Cents\',\n                 \'First Name.2\', \'Last Name.2\', \'3p_Dollar\', \'3p_Cents\']\nNY_CD_raw[\'state\'] = np.nan\nNY_CD_raw[\'town\'] = np.nan\nNY_CD_raw[\'occupation\'] = np.nan\nNY_CD = NY_CD_raw[[\'6p_Dollar\', \'6p_Cents\', \'6p_def_Dollar\', \'6p_def_Cents\',\'3p_Dollar\', \'3p_Cents\',\n                   \'town\', \'state\', \'occupation\']]\nCD_all = pd.concat([CD_all, NY_CD])\n'

In [15]:
"""#prepare loan dataset
NY_ASD_raw = pd.read_excel("../../Data/Post1790/NY/NY_1790_CD.xlsx",
                          header = 11, usecols = 'H, I, M, N, X, Y, AC, AD, AM, AN, AR, AS')
NY_ASD_raw.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents',
                     'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                     'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
NY_ASD_raw['state'] = np.nan
NY_ASD_raw['town'] = np.nan
NY_ASD_raw['occupation'] = np.nan
NY_ASD = NY_ASD_raw[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents','3p_Dollar', '3p_Cents',
                   'town', 'state', 'occupation']]
ASD_all = pd.concat([ASD_all, NY_ASD])
"""

'#prepare loan dataset\nNY_ASD_raw = pd.read_excel("../../Data/Post1790/NY/NY_1790_CD.xlsx",\n                          header = 11, usecols = \'H, I, M, N, X, Y, AC, AD, AM, AN, AR, AS\')\nNY_ASD_raw.columns = [\'First Name\', \'Last Name\', \'6p_Dollar\', \'6p_Cents\',\n                     \'First Name.1\', \'Last Name.1\', \'6p_def_Dollar\', \'6p_def_Cents\',\n                     \'First Name.2\', \'Last Name.2\', \'3p_Dollar\', \'3p_Cents\']\nNY_ASD_raw[\'state\'] = np.nan\nNY_ASD_raw[\'town\'] = np.nan\nNY_ASD_raw[\'occupation\'] = np.nan\nNY_ASD = NY_ASD_raw[[\'6p_Dollar\', \'6p_Cents\', \'6p_def_Dollar\', \'6p_def_Cents\',\'3p_Dollar\', \'3p_Cents\',\n                   \'town\', \'state\', \'occupation\']]\nASD_all = pd.concat([ASD_all, NY_ASD])\n'

### Pennsylvania

In [16]:
#prepare loan dataset
PA_CD_raw = pd.read_excel("../../Data/Post1790/PA/PA_post1790_CD.xlsx",
                      header = 11, usecols = 'G, H, I, J, K, L, M, U, V, W, X, Y, Z, AA, AJ, AK, AL, AM, AN, AO, AP')
PA_CD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                 'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                 'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
# unify occupation, town and state columns
PA_CD, change_df = combineCols(PA_CD_raw)
CD_all = pd.concat([CD_all, PA_CD])
change_df

town column has multiple unique entries
see table at end for new entries
reformatting state
occupation column has multiple unique entries
see table at end for new entries


,old,new,type
14,"{Carlisle Pennsylvania, Carlisle}",Carlisle Pennsylvania,town
39,"{Connecticut, Conecticutt}",Connecticut,town
40,"{New Castle Delaware, Newcastle Delaware}",New Castle Delaware,town
113,"{Philadelphia County, Philadelphia}",Philadelphia County,town
131,"{State of Delawere, State of Delaware}",State of Delawere,town
...,...,...,...
1143,"{Merchant, Esquire}",Merchant,occupation
1181,"{Shopkeeper, Shop Keeper}",Shop Keeper,occupation
1200,"{in trust for the Estate of Robert Dill, Trust...",in trust for the Estate of Robert Dill,occupation
1224,{Administrator to the Estate of William Barrel...,Administrator to the Estate of William Barrell...,occupation


### Rhode Island

In [17]:
#prepare loan dataset
RI_CD_raw = pd.read_excel("../../Data/Post1790/RI/T653_Rhode_Island_CD.xlsx",
                      header = 11, usecols = 'G, H, I, J, K, L, M, U, V, W, X, Y, Z, AA, AI, AJ, AK, AL, AM, AN, AO')
RI_CD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                     'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                     'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
RI_CD, change_df = combineCols(RI_CD_raw)
CD_all = pd.concat([CD_all, RI_CD])
change_df

town column has multiple unique entries
see table at end for new entries
reformatting state
reformatting occupation


,old,new,type
26,"{Gloucester, Glocester}",Gloucester,town
371,"{Charlestown, Chalestown}",Charlestown,town
435,"{North Kingston, North Kingstone}",North Kingstone,town
462,"{North Kingston, North Kingstown}",North Kingstown,town
499,"{Smithfeild, Smithfield}",Smithfeild,town


In [18]:
#prepare loan dataset
RI_ASD_raw = pd.read_excel("../../Data/Post1790/RI/T653_Rhode_Island_ASD.xlsx",
                          header = 11, usecols = 'H, I, J, K, L, N, O, X, Y, Z, AA, AB, AD, AE, AN, AO, AP, AQ, AR, AT, AU')
RI_ASD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                     'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                     'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
RI_ASD, change_df = combineCols(RI_ASD_raw)
ASD_all = pd.concat([CD_all, RI_ASD])
change_df

reformatting town
reformatting state
reformatting occupation


,old,new,type


### South Carolina

In [19]:
#prepare loan dataset
SC_CD_raw = pd.read_excel("../../Data/Post1790/SC/Post_1790_South_Carolina_CD.xlsx",
                      header = 11, usecols = 'D, E, F, G, H, M, N, S, T, U, V, W, AB, AC, AH, AI, AJ, AK, AL, AQ, AR')
SC_CD_raw.columns = ['First Name', 'Last Name', 'town1', 'state1', 'occupation1', '6p_Dollar', '6p_Cents',
                 'First Name.1', 'Last Name.1', 'town2', 'state2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                 'First Name.2', 'Last Name.2', 'town3', 'state3', 'occupation3', '3p_Dollar', '3p_Cents', ]
SC_CD, change_df = combineCols(SC_CD_raw)
CD_all = pd.concat([CD_all, SC_CD])
change_df

town column has multiple unique entries
see table at end for new entries
reformatting state
occupation column has multiple unique entries
see table at end for new entries


,old,new,type
50,"{Camden Planter, Camden}",Camden Planter,town
94,"{Pee Dee, Peedee}",Pee Dee,town
270,"{Long Cane, Long Cames}",Long Cames,town
292,"{Charleston, Charlestom}",Charleston,town
45,"{as Executor to Henry Coram, as Executor to Jo...",as Executor to John Couturier,occupation
50,"{Planter as Executor Ely Kershaw, as Executor ...",Planter as Executor Ely Kershaw,occupation
182,"{Executor Philip Hawkins, Philip Hawkins}",Executor Philip Hawkins,occupation
256,"{as Guardian to Mary Deborah L. Gowdey, Guardi...",as Guardian to Mary Deborah L. Gowdey,occupation
305,"{Merchants, Charleston Merchants}",Charleston Merchants,occupation
383,"{Physician, Assignee of James Simons}",Assignee of James Simons,occupation


In [20]:
#prepare loan dataset
SC_ASD_raw = pd.read_excel("../../Data/Post1790/SC/Post_1790_South_Carolina_ASD_transfers_removed.xlsx", header = 11,
                       usecols = 'D, E, F, G, H, M, N, O')
SC_ASD_raw.columns = ['First Name', 'Last Name', 'town', 'state', 'occupation', '6p_Dollar', '6p_def_Dollar','3p_Dollar']
for col in ['6p_', '6p_def_', '3p_']:
    SC_ASD_raw[col+'Cents'] = SC_ASD_raw[col+'Dollar'] - np.round(SC_ASD_raw[col+'Dollar'], 0)
    SC_ASD_raw[col+'Dollar'] = np.round(SC_ASD_raw['6p_Dollar'], 0)
SC_ASD = SC_ASD_raw[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents',
                     'town', 'state', 'occupation']]
ASD_all = pd.concat([ASD_all, SC_ASD])

### Virginia

In [21]:
"""
#prepare loan dataset
VA_CD_raw = pd.read_excel("../../Data/Post1790/VA/VA_CD.xlsx",
                      header = 11, usecols = 'H, I, K, L, U, V, X, Y, AH, AI, AK, AL')
VA_CD_raw.columns = ['First Name', 'Last Name', '6p_Dollar', '6p_Cents',
                 'First Name.1', 'Last Name.1', '6p_def_Dollar', '6p_def_Cents',
                 'First Name.2', 'Last Name.2', '3p_Dollar', '3p_Cents']
VA_CD_raw['state'] = np.nan
VA_CD_raw['town'] = np.nan
VA_CD_raw['occupation'] = np.nan
VA_CD = VA_CD_raw[['6p_Dollar', '6p_Cents', '6p_def_Dollar', '6p_def_Cents', '3p_Dollar', '3p_Cents',
                   'town', 'state', 'occupation']]
CD_all = pd.concat([CD_all, VA_CD])
"""

'\n#prepare loan dataset\nVA_CD_raw = pd.read_excel("../../Data/Post1790/VA/VA_CD.xlsx",\n                      header = 11, usecols = \'H, I, K, L, U, V, X, Y, AH, AI, AK, AL\')\nVA_CD_raw.columns = [\'First Name\', \'Last Name\', \'6p_Dollar\', \'6p_Cents\',\n                 \'First Name.1\', \'Last Name.1\', \'6p_def_Dollar\', \'6p_def_Cents\',\n                 \'First Name.2\', \'Last Name.2\', \'3p_Dollar\', \'3p_Cents\']\nVA_CD_raw[\'state\'] = np.nan\nVA_CD_raw[\'town\'] = np.nan\nVA_CD_raw[\'occupation\'] = np.nan\nVA_CD = VA_CD_raw[[\'6p_Dollar\', \'6p_Cents\', \'6p_def_Dollar\', \'6p_def_Cents\', \'3p_Dollar\', \'3p_Cents\',\n                   \'town\', \'state\', \'occupation\']]\nCD_all = pd.concat([CD_all, VA_CD])\n'

In [22]:
#prepare loan dataset
VA_ASD_raw = pd.read_excel("../../Data/Post1790/VA/VA_ASD.xlsx", header = 11,
                       usecols = 'D, E, F, G, N, O, U, V, W, X, AE, AF, AL, AM, AN, AO, AW, AX')
VA_ASD_raw.columns = ['First Name', 'Last Name', 'town1', 'occupation1', '6p_Dollar', '6p_Cents',
                  'First Name.1', 'Last Name.1', 'town2', 'occupation2', '6p_def_Dollar', '6p_def_Cents',
                  'First Name.2', 'Last Name.2', 'town3', 'occupation3', '3p_Dollar', '3p_Cents']
VA_ASD_raw['state1'] = np.nan
VA_ASD_raw['state2'] = np.nan
VA_ASD_raw['state3'] = np.nan
VA_ASD, change_df = combineCols(VA_ASD_raw)
ASD_all = pd.concat([ASD_all, VA_ASD])
change_df

town column has multiple unique entries
see table at end for new entries
reformatting state
occupation column has multiple unique entries
see table at end for new entries


,old,new,type
0,"{Town of Petersburg, The Town Petersburg}",The Town Petersburg,town
1,"{City of Richmond, Richmond}",City of Richmond,town
2,"{Dinwiddie County, Dinwiddie Country}",Dinwiddie Country,town
3,"{Town of Petersburg, The Town Petersburg, Town...",The Town Petersburg,town
4,"{Halifax County, Halifox County}",Halifax County,town
...,...,...,...
743,"{In trust & co I Ball, In trust for J Ball}",In trust & co I Ball,occupation
777,"{In trust for George Morrison London, Trust fo...",In trust for George Morrison London,occupation
831,"{In trust for J Robinson, In trust for Jn Robi...",In trust for Jn Robinson RB,occupation
875,"{Executors James Minor, Executors of James Min...",Exec James Minor Louisa Co,occupation


In [269]:
ASD_all

,6p_Dollar,6p_Cents,6p_def_Dollar,6p_def_Cents,3p_Dollar,3p_Cents,town,state,occupation
0,1064.0,75.0,532.0,37.0,508.0,51.0,Hartford,CT,Merchant
1,449.0,96.0,224.0,97.0,232.0,10.0,Bolton,CT,Farmer
2,154.0,20.0,77.0,10.0,192.0,NaN,Rhode Island,RI,Farmer
3,196.0,75.0,98.0,37.0,172.0,24.0,Hartford,CT,Merchant
4,53.0,58.0,26.0,79.0,67.0,6.0,Hartford,CT,Merchant
...,...,...,...,...,...,...,...,...,...
903,NaN,NaN,NaN,NaN,28.0,29.0,Richmond,NaN,NaN
904,NaN,NaN,NaN,NaN,2030.0,35.0,New York,NaN,NaN
905,954.0,43.0,477.0,21.0,715.0,82.0,North Carolina,NaN,Administrator of McKennie Sumner
906,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Mapping Town/City to Counties - INCOMPLETE
1. Connecticut: Referencing <a href = "https://ctstatelibrary.org/cttowns/counties">https://ctstatelibrary.org/cttowns/counties</a> I found that Huntington is now called Shelton and Chatham is now called East Hampton. The other two cases below are not mappable because those are not valid town names.
2. Georgia: Investigate more, very few counties

In [259]:
# fuzzy string matching function
def fuzzyMatch(unmatched_towns, towns, crosswalk, primary_dict, dict_matchcol = 'primary_city', initial = True, score_threshold = 85):
    if initial:
        print("\nFuzzy City name - county matches\n")
    else:
        print("\nFuzzy City name - county matches with string changes\n")

    for town in unmatched_towns:
        # extract best match
        match_tuple = process.extractOne(town, crosswalk[dict_matchcol])
        score = match_tuple[1]
        match = match_tuple[0]
        # if match above threshold, change + print match so we can hand check
        if score >= score_threshold:

            if dict_matchcol == 'primary_city':
                county = primary_dict[match]
            if dict_matchcol == 'county':
                county = match
            # add match, print out match
            if initial:
                print("{} -> {} in {}".format(town, match, county))
                town_index = towns[towns['town'] == town].index
                towns.loc[town_index, 'county'] = county
            else:
                original_town = towns[towns['town2'] == town]['town'].tolist()
                print("{} (new name: {}) -> {} in {}".format(original_town, town, match, county))
                town_index = towns[towns['town'].apply(lambda x: x in original_town)].index
                towns.loc[town_index, 'county'] = [county] * len(town_index)
    return towns

In [254]:
def directTownMatch(state_cw, towns, col = 'primary_city', towncol = 'town'):
    print("Direct City name - county matches\n")
    # match towns directly based off crosswalk
    primary_dict = dict(zip(state_cw[col],state_cw['county']))
    if col == 'primary_city':
        towns['county'] = towns[towncol].apply(lambda x: primary_dict.get(x, np.nan))
    if col == 'acceptable_cities':
        for ind in towns.index:
            town = towns.loc[ind, 'town']
            county = state_cw[state_cw[col].apply(lambda x: town in x if not pd.isnull(x) else False)]['county'].tolist()
            if len(county)>0:
                towns.loc[ind, 'county'] = county[0]
    t = towns[towns['county'].apply(lambda x: not pd.isnull(x))]
    if towncol == 'town':
        for tn, cty in zip(t['town'], t['county']):
            print("{} was matched to {} directly using the crosswalk".format(tn, cty))
    if towncol == 'town2':
        for tn, tn_og, cty in zip(t['town2'], t['town'], t['county']):
            print("{} (original: {}) was matched to {} directly using the crosswalk".format(tn, tn_og, cty))
    return primary_dict, towns

In [255]:
def directCountyMatch(state_cw, towns, towncol = 'town'):
    print("\nSome city names are actually county names")
    if towncol == 'town':
        print("Direct City (county) name - county matches\n")
    if towncol == 'town2':
        print("Direct City (county) name with string changes - county matches\n")
    # some own names are actually counties
    # match towns based off whether town name is actually county name in crosswalk
    counties = state_cw['county'].unique()
    nanindex = towns[towns['county'].apply(lambda x: pd.isnull(x))].index
    towns.loc[nanindex, 'county'] = towns.loc[nanindex, towncol].apply(lambda x: x if x in counties.tolist() else np.nan)
    towns2 = towns.loc[nanindex]
    nanindex2 = towns2[towns2['county'].apply(lambda x: not pd.isnull(x))].index
    for t, c in zip(towns2.loc[nanindex2, towncol], towns2.loc[nanindex2, 'county']):
        print("{} was matched to {} using the crosswalk".format(t, c))
    return towns

In [105]:
city_county_cw = pd.read_excel('../../Data/zip_code_database.xls')[['primary_city', 'acceptable_cities',
                                                                    'unacceptable_cities', 'county', 'state']]

In [268]:
final_cw = pd.DataFrame(columns = ['town', 'county', 'state'])
list_of_states = ['CT', 'GA', 'MD', 'NC', 'NH', 'NJ',
                  #'NY',
                  'PA']
for state in list_of_states:
    print("\n{} MATCHING \n".format(state))
    # create list of towns for each state
    towns = CD_all[CD_all['state'] == state][['town']].drop_duplicates()
    towns = towns[towns['town'].apply(lambda x: not pd.isnull(x))]
    # state crosswalk
    state_cw = city_county_cw[city_county_cw['state'] == state]

    # try initial direct town match
    primary_dict, towns = directTownMatch(state_cw, towns)
    if state == 'CT':
        # try fuzzy string match merge  - match town column with crosswalk town column
        unmatched_towns1 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town']
        towns = fuzzyMatch(unmatched_towns1, towns, state_cw, primary_dict)

        # modify towns that changed names - see note at very top
        # then retry fuzzy string match with new town names
        towns['town2'] = towns['town'].apply(lambda x: x.replace('Huntington', 'Shelton').replace('Chatham', 'East Hampton'))
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol ='primary_city', initial = False)
        towns.drop('town2', axis = 1, inplace = True)

    if state == 'GA':
        # some town names are actually county names, try direct town-county match
        towns = directCountyMatch(state_cw, towns)

    if state == 'MD':
        # some town names are actually county names, try direct town-county match
        towns = directCountyMatch(state_cw, towns)

        # some town names are actually county names, try direct town-county match - but with renamed strings
        towns['town2'] = towns['town'].apply(lambda x: x.replace('Maryland', '').replace('Co ', 'County').strip())
        towns = directCountyMatch(state_cw, towns, towncol = 'town2')

        # try fuzzy string match with county names (compare county names to crosswalk county names)
        towns['town2'] = towns['town'].apply(lambda x: x.replace('Maryland', '').replace('Co ', 'County').strip())
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, dict_matchcol = 'county', initial = False, score_threshold = 86)

        # try fuzzy string match with town names + string modifications (compare town names to crosswalk town names)
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, initial = False)

        # correct a matching
        towns['county'] = towns['county'].apply(lambda x: x.replace('City', 'County') if not pd.isnull(x) else x)

    if state == 'NC':
        # some town names are actually county names, try direct town-county match
        towns = directCountyMatch(state_cw, towns)

        # try fuzzy string match merge - match town column with crosswalk town column
        towns['town2'] = towns['town'].apply(lambda x: x.replace('North Carolina', '').replace('Tarborugh', 'Tarboro').strip())
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, initial = False)

        # manually fixing bug - already matched name won't drop
        towns.loc[towns[towns['town'] == 'Halifax'].index, 'county'] = 'Halifax County'
    if state == 'NH':
        # try fuzzy string match merge - match town column with crosswalk town column
        towns['town2'] = towns['town'].apply(lambda x: x.replace('State', '').replace('New Hampshire', '').replace('of ','').strip())
        towns['town2'] = towns['town2'].apply(lambda x: x.replace('Rockingham', 'Rockingham County').strip())

        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, initial = False)

        null_ind = towns[towns['county'].apply(lambda x: pd.isnull(x))].index
        pdict, tn = directTownMatch(state_cw, towns.loc[null_ind], 'acceptable_cities')
        towns.loc[null_ind] = tn

        # some town names are actually county names, try direct town-county match
        towns = directCountyMatch(state_cw, towns, 'town2')

        # manual fixes
        towns.loc[towns[towns['town'] == 'Brintwood'].index, 'county'] = 'Rockingham County'
        towns.loc[towns[towns['town'] == 'Portsmouth New Hampshire'].index, 'county'] = 'Portsmouth County'
    if state == 'NJ':
        # try fuzzy string match merge - match town column with crosswalk town column
        towns['town2'] = towns['town'].apply(lambda x: x.replace('New Jersey', '').strip())
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, initial = False)

        # some town names are actually county names, try direct town-county match
        towns = directCountyMatch(state_cw, towns, 'town2')
    if state == 'PA':
        towns = directCountyMatch(state_cw, towns)

        towns['town2'] = towns['town'].apply(lambda x: x.replace('Co ', 'County').replace('Delaware', 'Delaware County').strip())
        towns['town2'] = towns['town2'].apply(lambda x: x.replace('Pennsylvania', '').replace('County County','County').strip())

        # Blockley is now defunct
        towns['town2'] = towns['town2'].apply(lambda x: x.replace('Blockley', 'Philadelphia').strip())
        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = directCountyMatch(state_cw, towns, 'town2')

        unmatched_towns2 = towns[towns['county'].apply(lambda x: pd.isnull(x))]['town2']
        towns = fuzzyMatch(unmatched_towns2, towns, state_cw, primary_dict, initial = False)

    # print out all unmatched names
    print("\nFinal Unmatched Names\n")
    t = towns[towns['county'].apply(lambda x: pd.isnull(x))]
    for tn in t['town']:
        print("{} was unable to be matched".format(tn))

#Charleston South Carolina (new name: Charleston South Carolina) -> South Heights in Beaver County
#Delaware County Pennsylvania (new name: Delaware) -> Delaware Water Gap in Monroe County
#Burlington New Jersey
#Northumberland County Virginia
#Chester Co
    #towns = towns[towns['county'].apply(lambda x: not pd.isnull(x))]
    #towns['state'] = state
    #final_cw = pd.concat([final_cw, towns])


CT MATCHING 

Direct City name - county matches

Hartford was matched to Hartford County directly using the crosswalk
Bolton was matched to Tolland County directly using the crosswalk
Wethersfield was matched to Hartford County directly using the crosswalk
New Haven was matched to New Haven County directly using the crosswalk
Farmington was matched to Hartford County directly using the crosswalk
New London was matched to New London County directly using the crosswalk
Cornwall was matched to Litchfield County directly using the crosswalk
Stamford was matched to Fairfield County directly using the crosswalk
East Hartford was matched to Hartford County directly using the crosswalk
Bristol was matched to Hartford County directly using the crosswalk
Lebanon was matched to New London County directly using the crosswalk
Windsor was matched to Hartford County directly using the crosswalk
Suffield was matched to Hartford County directly using the crosswalk
Berlin was matched to Hartford County

In [267]:
Tulpehocken berks county


,primary_city,acceptable_cities,unacceptable_cities,county,state


In [261]:
towns[towns['county'].isnull()]

,town,county,town2
149,Pennsylvania,NaN,
101,Blockley,NaN,Blockley
108,W Callisters Town,NaN,W Callisters Town
177,Tulpehocken,NaN,Tulpehocken
215,Armstrong,NaN,Armstrong
293,Northumb Country,NaN,Northumb Country
343,Northan Liberties,NaN,Northan Liberties
501,Borden Town,NaN,Borden Town
539,St Eustalia,NaN,St Eustalia
583,Passyunk,NaN,Passyunk
